In [1]:
# CIS 545

# Homework 3, Part I: Spark

## Spark Setup

Please make sure the Jupyter menu above shows "PySpark" at the upper right, and not "Python 3".  If it says "Python 3" then go to the *Kernel* menu and choose *PySpark*.


## Step 1

### Breadth-first Search over the NotreDame Graph

In the previous assignment, you had built several primitives, including breadth-first search, to compute over the Yelp social graph.  Some of your computations were limited by the amount of computational resources available.  We will now try to explore the broader graph.

### Read **prior** to starting assignment

You will see some of the following in the cells below, and you should treat them as follows:
- `[hw3-cellname]` This is a cell label. You should not delete these.
- For the CIS 545 test case cells with score additions, those are just for padding, they have no purpose but to let the autograder run more smoothly! The `score` variable **is not** what you will get in an autograder score
- try/catch: This is for the name of your dataset. Please read the instructions carefully.

Hints: For early testing of your solutions to Step 2, you should probably just load a subset of thisl file into `graph_sdf` to validate your solutions.  Later go back, add the contents of the other files to graph_sdf, and re-run your solutions.  Make sure you rerun your code with the full `graph_sdf` before submitting. 

### Google Code Spark Initialization

These codes are a setup for your reference.

In [2]:
# [hw3-import]
# Run this cell to import the various PySpark and OS operations

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import * 
import pyspark.sql.functions as F
import pyspark.sql.functions as f

import pandas as pd
import logging
import os

In [3]:
# [hw3-sparkcheck]
# Just a check to make sure you have the pyspark kernel.

try:
    if(spark == None): 
        raise ValueError("PySpark Kernel not ready! ")
        # The following sentence only run on local machine. 
#         spark = SparkSession.builder.appName('Graphs-HW3').getOrCreate()
except NameError:
    raise ValueError("PySpark Kernel not ready! ")
    # The following sentence only run on local machine. 
#     spark = SparkSession.builder.appName('Graphs-HW3').getOrCreate()

In [4]:
# This is a padding cell for Spark operation

In [5]:
# [hw3-pad]
# This cell is for grading purposes only!

score = 0 # for manual grading

print("[CIS 545 Homework 3] Test Case Padding")

[CIS 545 Homework 3] Test Case Padding


### Step 1.1. Loading the Data

Let’s read our graph data from NotreDame, which forms a directed graph.  Here, the set of nodes is also not specified; the assumption is that the only nodes that matter are linked to other nodes, and thus their IDs will appear in the set of edges.  For example, to load the file `input.txt` into a Spark DataFrame, you can use lines like the following. Note that we aren't running just `.txt` files in this assignment!

```
# Read lines from the text file, from Google Storage bucket my-bucket
input_sdf = spark.read.format("txt").load("gs://my-bucket/mypath/input.txt")
```

We’ll use the suffix `_sdf` to represent “Spark DataFrame,” much as we used `_sdf` to denote a Spark DataFrame in Homework 2. You will load a `ndgraph_sdf` table from the `web-NotreDame.csv` file. 

Downloading these files may take a while, so don’t worry. See if you can further add a function call (e.g., to selectExpr()) to rename the edges to `from_node` and `to_node`, to convert these to integers, respectively. Remove duplicates to create `graph_sdf` with all data.

Please name the tables as we have described in this assignment. This helps with our grading and test cases.

In [6]:
# [hw3-filepath]
# We'll use the variable dataset to store the filepath for your csv

try:
    # This is how you should load your data
    dataset = 'web-NotreDame.csv'
except:
    # Note that this is not going to be a valid link until we grade!
    dataset = 'https://s3.amazonaws.com/cis545-hw3data/web-NotreDame.csv'

Above, note that we stored `dataset` as the name of your CSV. You should use the variable dataset below when importing, in lieu of a filepath directly. If you need to change it, you may change the allocation of the variable below.

Note that you should not change the S3 link, as doing so (or not using the variable `dataset` below) will lead to issues autograding your homework.

In [7]:
# [hw3-importdata]
# Import your dataset

# YOUR CODE HERE
ndgraph_sdf = spark.read.format("csv").option("header", "true").load('gs://bucket-hw3-1/notebooks/' +dataset)


The following cells are used for grading and data entry purposes, if necessary. Please do not edit the cells (although it is allowed to be edited). If you must, add cells **below** them.

In [8]:
# [hw3-dataload]
# Do not edit!

You may edit cells below **here**.

In [9]:
# [hw3-printschema]
# View the schema presented 

ndgraph_sdf.printSchema()

root
 |-- from: string (nullable = true)
 |-- to: string (nullable = true)



In [10]:
# [hw3-showndgraph]
# View the output of ndgraph. You should see your correct columns

ndgraph_sdf.show()

+----+---+
|from| to|
+----+---+
|   0|  0|
|   0|  1|
|   0|  2|
|   0|  3|
|   0|  4|
|   0|  5|
|   0|  6|
|   0|  7|
|   0|  8|
|   0|  9|
|   0| 10|
|   0| 11|
|   0| 12|
|   0| 13|
|   0| 14|
|   0| 15|
|   0| 16|
|   1|  0|
|   1|  7|
|   1| 17|
+----+---+
only showing top 20 rows



In [11]:
# [test-colcount]
# [CIS 545 Homework 3] Test Case (test case pad below!)

assert (len(ndgraph_sdf.columns) == 2)


In [12]:
# [CIS 545 Test Cases] (2 pts)
print("[CIS 545 Homework 3] Test Case - 2 points")
score = score + 2

[CIS 545 Homework 3] Test Case - 2 points


In [13]:
# [test-colset]
# [CIS 545 Homework 3] Test Case


In [14]:
# [CIS 545 Test Cases] (5 pts)
print("[CIS 545 Homework 3] Test Case - 5 points")
score = score + 5

[CIS 545 Homework 3] Test Case - 5 points


You will want to create a `graph_sdf` and save it in the Spark TempView so that you can call your transitive closure function on it here.

Hints: For early testing of your solutions to Step 2, you should probably just load a subset of this file into `graph_sdf` to validate your solutions.  Later go back, add the contents of the other files to graph_sdf, and re-run your solutions.  Make sure you rerun your code with the full `graph_sdf` before submitting. 

In [15]:
# [hw3-pysparkF]
# Import PySpark types and functions

from pyspark.sql.types import *
import pyspark.sql.functions as F

In [16]:
# [hw3-graph]
# Create graph_sdf in ndgraph_sdf and create a SQL table if you decide to use it later.
#See if you can further add a function call (e.g., to selectExpr()) to rename the edges to from_node and to_node, 
#to convert these to integers, respectively. Remove duplicates to create graph_sdf with all data.
# val df2 = df.selectExpr("cast(year as int) year")
graph_sdf = ndgraph_sdf.selectExpr("cast(from as int) from_node", "cast(to as int) to_node")
graph_sdf=graph_sdf.dropDuplicates().na.drop()
graph_sdf.createOrReplaceTempView('graph_sql')
graph_sdf.printSchema()
#print(ndgraph_sdf.count(),graph_sdf.count())

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)



In [17]:
# [hw3-padding]
# Just a padding cell
print("[CIS 545 Homework 3] Padding")

[CIS 545 Homework 3] Padding


In [18]:
# [hw3-cache]
# You should now cache for efficiency
graph_sdf.cache()

DataFrame[from_node: int, to_node: int]

### Step 1.2. Transitive Closure of the Graph

Now we would like to do the following: given a set of nodes, compute the set of all nodes reachable from these nodes.  This can be obtained via a type of transitive closure computation. Google it if you don't know Transitive Closure. 

Define a function `transitive_closure(graph_sdf, origins_sdf, depth)` that returns a Spark DataFrame.

The result should be the set of all nodes from the `input graph_sdf` that are reachable via graph edges from the set of `origins_sdf`, in at most depth iterations (hops).  Both origins_sdf and the returned result should be DataFrames with a single attribute called node.  

You should treat the edges in the `graph_sdf` as directed edges!  You should iterate until you have either hit the maximum depth or the set of newly discovered (frontier) nodes is empty.

Hints: this resembles your BFS algorithm from HW2, but you should take advantage of the opportunities to optimize.  Both the graph and the various node sets can easily have duplicates; you should make heavy use of duplicate removal (and repartition and cache) since you are only computing the set of reachable nodes. Also, to quickly check whether a DataFrame is empty, you can use something similar to the following. 

(You are not needed to update the following code block. Please start your code in the block containing `transitive_closure`

In [19]:
sc.defaultParallelism

4

In [20]:
# [hw3-sdfempty]
# This code block may be used in transitive_closure

def sdf_is_empty(sdf):
    try:
        sdf.take(1)
        return False
    except:
        return True

Please insert your code for `transitive_closure` below. You may find repartitioning or caching valuable in speeding up your queries.

In [21]:
sc.version

'2.3.2'

In [22]:
def transitive_closure(graph, origin, depth):
    result_sdf = origin.cache()
    frontier_sdf = result_sdf.cache()  
    i=0
    while True:
        if i==depth or frontier_sdf.count()==0:
            return result_sdf
        i=i+1
        frontier_sdf = frontier_sdf.join(graph, frontier_sdf.node==graph.from_node,'left').select('to_node').distinct().na.drop().withColumnRenamed('to_node','node').repartition(500).cache()
        frontier_sdf = frontier_sdf.join(result_sdf, frontier_sdf.node==result_sdf.node, 'leftanti').distinct().na.drop().repartition(500).cache()
        result_sdf = result_sdf.union(frontier_sdf).distinct().na.drop().repartition(500).cache()
        print(result_sdf.count())
#         print(frontier_sdf.count())
#         frontier_sdf.show(10)


### Step 1.3 Testing transitive closure

Compute a Spark DataFrame called `nodes_sdf` as follows:
* Select `from_node` id 113, 114, 115 and 116
* Rename the column to `node`


In [23]:
# [hw3-nodes]
# Create nodes_sdf as described above

nodes_sdf = graph_sdf.select('from_node').where(graph_sdf.from_node.isin([113,114,115,116])).distinct().na.drop().withColumnRenamed('from_node','node').cache()


In [24]:
# [hw3-nodeshow]
# View nodes_sdf

nodes_sdf.show()

+----+
|node|
+----+
| 115|
| 114|
| 113|
| 116|
+----+



In [25]:
# [test-nodes]
# Testing nodes setup


In [26]:
# [CIS 545 Test Cases] (5 pts)
print("[CIS 545 Homework 3] Test Case - 5 points")
score = score + 5

[CIS 545 Homework 3] Test Case - 5 points


Compute a Spark DataFrame called `reachable_sdf` with the results of `transitive_closure(graph_sdf, nodes_sdf, 3)` over the NotreDame dataset. 

In [27]:
# [hw3-reachable]
# Create reachable_sdf using transitive closure

#len_edge = graph_sdf.select("from_node").distinct().count()-1    
reachable_sdf=transitive_closure(graph_sdf, nodes_sdf, 3) 
reachable_sdf.show()

38
170
426
+-----+
| node|
+-----+
|12296|
|12193|
|12533|
| 3244|
| 1122|
|12167|
| 1132|
|12195|
|12174|
| 3298|
| 8392|
| 1123|
|12330|
|    9|
|12175|
|12290|
|  754|
|14416|
|12216|
| 4390|
+-----+
only showing top 20 rows



In [28]:
# [test-getresult]
# This is a no-points test case, making sure we get a result
assert (reachable_sdf.take(1)[0].node != None)

In [29]:
# [hw3-padding]
# Just a padding cell

print("[CIS 545 Homework 3] Padding")

[CIS 545 Homework 3] Padding


Run the code cell to call `show()` and `count` on `reachable_sdf`

In [30]:
# [hw3-showreach]
# Show reachacble_sdf

reachable_sdf.show(20)

+-----+
| node|
+-----+
|12296|
|12193|
|12533|
| 3244|
| 1122|
|12167|
| 1132|
|12195|
|12174|
| 3298|
| 8392|
| 1123|
|12330|
|    9|
|12175|
|12290|
|  754|
|14416|
|12216|
| 4390|
+-----+
only showing top 20 rows



In [31]:
# [hw3-count]
# Show the count of reachable_sdf

reachable_count = reachable_sdf.count()
reachable_count

426

In [32]:
# [test-reachable1]
# Tests on reachable_sdf


In [33]:
# [CIS 545 Test Cases] (5 pts)
print("[CIS 545 Homework 3] Test Case - 5 points")
score = score + 5

[CIS 545 Homework 3] Test Case - 5 points


In [34]:
# [test-reachable1]
# Tests on reachable_sdf


In [35]:
# [CIS 545 Test Cases] (5 pts)
print("[CIS 545 Homework 3] Test Case - 5 points")
score = score + 5

[CIS 545 Homework 3] Test Case - 5 points
